In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nfl_data_py as nfl

In [79]:
base_pbp = nfl.import_pbp_data([2023], downcast=True, cache=False, alt_path=None)

2023 done.
Downcasting floats.


In [80]:
pd.set_option('display.max_columns', None)

In [81]:
pd.set_option('display.max_seq_items', None)

In [82]:
base_pbp.columns

Index(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       'side_of_field', 'yardline_100', 'game_date',
       'quarter_seconds_remaining', 'half_seconds_remaining',
       'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp',
       'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet',
       'desc', 'play_type', 'yards_gained', 'shotgun', 'no_huddle',
       'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length',
       'pass_location', 'air_yards', 'yards_after_catch', 'run_location',
       'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result',
       'two_point_conv_result', 'home_timeouts_remaining',
       'away_timeouts_remaining', 'timeout', 'timeout_team', 'td_team',
       'td_player_name', 'td_player_id', 'posteam_timeouts_remaining',
       'defteam_timeouts_remaining', 'total_home_score', 'total_away_score',
       'posteam_score'

In [83]:
# Step 2: Extract unique teams
teams = set(base_pbp['home_team'].unique()).union(set(base_pbp['away_team'].unique()))

# Step 3: Assign unique team_id to each team
team_id_mapping = {team: i + 1 for i, team in enumerate(sorted(teams))}  # Assigning sequential IDs starting from 1

# Step 4: Add home_team_id and away_team_id to the pbp data
base_pbp['home_team_id'] = base_pbp['home_team'].map(team_id_mapping)
base_pbp['away_team_id'] = base_pbp['away_team'].map(team_id_mapping)

# Display the updated pbp data
print(base_pbp)

       play_id          game_id old_game_id home_team away_team season_type  \
0          1.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
1         39.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
2         55.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
3         77.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
4        102.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
...        ...              ...         ...       ...       ...         ...   
49453   4160.0   2023_21_KC_BAL  2024012800       BAL        KC        POST   
49454   4192.0   2023_21_KC_BAL  2024012800       BAL        KC        POST   
49455   4221.0   2023_21_KC_BAL  2024012800       BAL        KC        POST   
49456   4243.0   2023_21_KC_BAL  2024012800       BAL        KC        POST   
49457   4265.0   2023_21_KC_BAL  2024012800       BAL        KC        POST   

       week posteam posteam_type defteam side_of_fi

C:\Users\bhugg\AppData\Local\Temp\ipykernel_18612\1391511132.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base_pbp['home_team_id'] = base_pbp['home_team'].map(team_id_mapping)
C:\Users\bhugg\AppData\Local\Temp\ipykernel_18612\1391511132.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base_pbp['away_team_id'] = base_pbp['away_team'].map(team_id_mapping)


In [84]:
# Step 1: Calculate average home_team_epa and average away_team_epa
average_home_team_epa = base_pbp.groupby('home_team')['epa'].mean()
average_away_team_epa = base_pbp.groupby('away_team')['epa'].mean()

# Step 2: Merge average home_team_epa and average away_team_epa with base_pbp
base_pbp = base_pbp.merge(average_home_team_epa.rename('average_home_team_epa'), left_on='home_team', right_index=True)
base_pbp = base_pbp.merge(average_away_team_epa.rename('average_away_team_epa'), left_on='away_team', right_index=True)

In [85]:
firsto = base_pbp[base_pbp['game_id'] == '2023_01_ARI_WAS']

In [86]:
gamble_check = firsto[['posteam', 'average_home_team_epa', 'average_away_team_epa', 'home_team_id', 'home_team', 'away_team_id', 'away_team', 'game_id', 'game_seconds_remaining', 'result', 'total', 'spread_line','total_line','home_wp', 'away_wp', 'safety', 'fumble', 'interception', 'qb_hit', 'total_home_score', 'total_away_score', 'score_differential']]
gamble_check.value_counts()

#use first for all these columns in aggregation

posteam  average_home_team_epa  average_away_team_epa  home_team_id  home_team  away_team_id  away_team  game_id          game_seconds_remaining  result  total  spread_line  total_line  home_wp   away_wp   safety  fumble  interception  qb_hit  total_home_score  total_away_score  score_differential
WAS      -0.012497              -0.0261                32            WAS        1             ARI        2023_01_ARI_WAS  3600.0                  4       36     7.0          38.0        0.546262  0.453738  0.0     0.0     0.0           0.0     0.0               0.0                0.0                  2
ARI      -0.012497              -0.0261                32            WAS        1             ARI        2023_01_ARI_WAS  2955.0                  4       36     7.0          38.0        0.697826  0.302174  0.0     0.0     0.0           0.0     7.0               0.0               -7.0                  2
                                                                                             

In [87]:
gamble_check.sort_values(by = 'game_seconds_remaining', ascending = False)

,posteam,average_home_team_epa,average_away_team_epa,home_team_id,home_team,away_team_id,away_team,game_id,game_seconds_remaining,result,total,spread_line,total_line,home_wp,away_wp,safety,fumble,interception,qb_hit,total_home_score,total_away_score,score_differential
0,None,-0.012497,-0.0261,32,WAS,1,ARI,2023_01_ARI_WAS,3600.0,4,36,7.0,38.0,0.546262,0.453738,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1,WAS,-0.012497,-0.0261,32,WAS,1,ARI,2023_01_ARI_WAS,3600.0,4,36,7.0,38.0,0.546262,0.453738,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,WAS,-0.012497,-0.0261,32,WAS,1,ARI,2023_01_ARI_WAS,3600.0,4,36,7.0,38.0,0.546262,0.453738,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,WAS,-0.012497,-0.0261,32,WAS,1,ARI,2023_01_ARI_WAS,3570.0,4,36,7.0,38.0,0.539621,0.460379,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,WAS,-0.012497,-0.0261,32,WAS,1,ARI,2023_01_ARI_WAS,3535.0,4,36,7.0,38.0,0.555987,0.444013,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,WAS,-0.012497,-0.0261,32,WAS,1,ARI,2023_01_ARI_WAS,60.0,4,36,7.0,38.0,0.973957,0.026043,0.0,0.0,0.0,0.0,20.0,16.0,4.0
171,WAS,-0.012497,-0.0261,32,WAS,1,ARI,2023_01_ARI_WAS,52.0,4,36,7.0,38.0,0.975044,0.024956,0.0,0.0,0.0,0.0,20.0,16.0,4.0
172,WAS,-0.012497,-0.0261,32,WAS,1,ARI,2023_01_ARI_WAS,7.0,4,36,7.0,38.0,0.964604,0.035396,0.0,0.0,0.0,0.0,20.0,16.0,4.0
173,WAS,-0.012497,-0.0261,32,WAS,1,ARI,2023_01_ARI_WAS,7.0,4,36,7.0,38.0,0.954844,0.045156,0.0,0.0,0.0,0.0,20.0,16.0,4.0


In [88]:
agg_data = {
    'home_team_id': gamble_check['home_team_id'].iloc[0],
    'home_team': gamble_check['home_team'].iloc[0],
    'away_team_id': gamble_check['away_team_id'].iloc[0],
    'away_team': gamble_check['away_team'].iloc[0],
    'game_id': gamble_check['game_id'].iloc[0],
    'total': gamble_check['total'].iloc[0],
    'home_spread': gamble_check['spread_line'].iloc[0],
    'total_line': gamble_check['total_line'].iloc[0],
    'average_home_team_epa': gamble_check['average_home_team_epa'].mean(),
    'average_away_team_epa': gamble_check['average_away_team_epa'].mean(),
    'home_wp': gamble_check['home_wp'].iloc[0],
    'away_wp': gamble_check['away_wp'].iloc[0],
    'home_fumbles': gamble_check[gamble_check['posteam'] == gamble_check['home_team']]['fumble'].sum(),
    'away_fumbles': gamble_check[gamble_check['posteam'] == gamble_check['away_team']]['fumble'].sum(),
    'home_interceptions': gamble_check[gamble_check['posteam'] == gamble_check['home_team']]['interception'].sum(),
    'away_interceptions': gamble_check[gamble_check['posteam'] == gamble_check['away_team']]['interception'].sum(),
    'home_safeties': gamble_check[gamble_check['posteam'] == gamble_check['home_team']]['safety'].sum(),
    'away_safeties': gamble_check[gamble_check['posteam'] == gamble_check['away_team']]['safety'].sum(),
    'home_qb_hits': gamble_check[gamble_check['posteam'] == gamble_check['home_team']]['qb_hit'].sum(),
    'away_qb_hits': gamble_check[gamble_check['posteam'] == gamble_check['away_team']]['qb_hit'].sum(),
    'total_home_score': gamble_check['total_home_score'].iloc[-1],
    'total_away_score': gamble_check['total_away_score'].iloc[-1],
    'score_differential': gamble_check['score_differential'].iloc[-1]
}

# Convert the aggregated data to DataFrame
agg_df = pd.DataFrame(agg_data, index=[0])

In [89]:
agg_df['away_spread'] = agg_df['home_spread'] * -1

In [90]:
agg_df

,home_team_id,home_team,away_team_id,away_team,game_id,total,home_spread,total_line,average_home_team_epa,average_away_team_epa,home_wp,away_wp,home_fumbles,away_fumbles,home_interceptions,away_interceptions,home_safeties,away_safeties,home_qb_hits,away_qb_hits,total_home_score,total_away_score,score_differential,away_spread
0,32,WAS,1,ARI,2023_01_ARI_WAS,36,7.0,38.0,-0.012497,-0.0261,0.546262,0.453738,2.0,2.0,1.0,0.0,0.0,0.0,6.0,5.0,20.0,16.0,4.0,-7.0
